In [46]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import random

In [18]:
train_df = pd.read_csv("./data/jigsaw/train.csv")
test_df = pd.read_csv("./data/jigsaw/test.csv")
test_labels_df = pd.read_csv("./data/jigsaw/test_labels.csv")
identities = pd.read_csv("./data/adjectives_people.txt")

In [19]:
test_df = pd.merge(test_df, test_labels_df)

In [20]:
train_df_short = train_df.drop(train_df.columns[3:8], axis=1)
train_df_short

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0


In [21]:
test_df_short = test_df.drop(test_df.columns[3:8], axis=1)

In [125]:
identity_array = identities.to_numpy()
identity_array = np.append(identity_array, 'lesbian')
identity_list = identity_array.tolist()

In [126]:
identity_list.remove('african american')
identity_list.remove('middle aged')
identity_list.remove('middle eastern')
eval_ids = []
eval_ids.append('african american')
eval_ids.append('middle aged')
eval_ids.append('middle eastern')
for i in range(12):
    rand_num = random.randint(0,len(identity_list))
    ident = identity_list.pop(rand_num)
    eval_ids.append(ident)
    
len(identity_list)
eval_ids

['african american',
 'middle aged',
 'middle eastern',
 'asian',
 'sikh',
 'latinx',
 'japanese',
 'black',
 'homosexual',
 'mexican',
 'catholic',
 'paralyzed',
 'white',
 'bisexual',
 'chinese']

In [164]:
with open("./data/train_identities.txt", "w") as output:
    for i in identity_list:
        output.write(str(i) + "\n")
    
with open("./data/test_identities.txt", "w") as output:
    for i in eval_ids:
        output.write(str(i) + "\n")

In [23]:
for row_index, row in tqdm(enumerate(train_df_short.itertuples())):
    for index, identity in enumerate(identity_list):
        if identity in row[2]:
            train_df_short.at[row_index, identity] = 1
        else:
            train_df_short.at[row_index, identity] = 0
#print(train_df_short)

159571it [01:10, 2265.54it/s]


In [24]:
# test df add 1s and 0s
for row_index, row in tqdm(enumerate(test_df_short.itertuples())):
    for index, identity in enumerate(identity_list):
        if identity in row[2]:
            test_df_short.at[row_index, identity] = 1
        else:
            test_df_short.at[row_index, identity] = 0

153164it [01:04, 2357.30it/s]


In [26]:
test_df_short.to_csv(Path("./data/test_df.csv"))
train_df_short.to_csv(Path("./data/train_df.csv"))

In [27]:
test_df_short = pd.read_csv("./data/test_df_short.csv")
train_df_short = pd.read_csv("./data/train_df.csv")

In [28]:
# for training data set:
# id - sentence - toxic - 50 cols with 0 if identity is not there, 1 if identity is there

In [179]:
train_identities_raw = pd.read_csv("./data/train_identities.txt")
test_identities = pd.read_csv("./data/test_identities.txt")

train_identities_listed = train_identities_raw.to_numpy().tolist()

train_identities = []
for i in train_identities_listed:
    train_identities.append(i[0])

# Blindness

In [180]:
# Adding identity column to train_df_short (either works I think)
for row_index, row in tqdm(enumerate(train_df_short.itertuples())):
    train_df_short.at[row_index, "identity"] = 0
    for i in range(4,54):
        if row[i] == 1:
            train_df_short.at[row_index, "identity"] = 1

159571it [00:03, 40412.77it/s]


In [181]:
# Adding identity column to train_df_short (either works I think)
for row_index in tqdm(range(len(train_df_short))):
    train_df_short.at[row_index, "identity"] = max([train_df_short.at[row_index, identity] for identity in train_identities])
    

100%|██████████| 159571/159571 [00:25<00:00, 6239.20it/s]


In [182]:
# Creating train_df_blind by dropping all identiy columns from train_df_short
train_df_blind = train_df_short.drop(columns = train_identities)

In [183]:
# Replacing identities in comment text with an identity token
token = "identity"
for row_index in tqdm(range(len(train_df_blind))):
    if train_df_short.at[row_index, "identity"] == 1:
        for identity in train_identities:
            if train_df_short.at[row_index, identity] == 1 :
                train_df_blind.at[row_index, "comment_text"] = train_df_blind.at[row_index, "comment_text"].replace(identity, token)

100%|██████████| 159571/159571 [00:03<00:00, 42759.03it/s]


In [184]:
train_df_blind.to_csv(Path("./data/train_df_blind.csv"))

# Augmentation

In [190]:
train_df_identities = train_df_short[train_df_short.identity==1]

In [191]:
train_df_identities = train_df_identities.reset_index()

In [199]:
comment_list = []
toxic_list = []
augmented = []

for row_index in tqdm(range(len(train_df_identities))):
    for identity in train_identities:
        if train_df_identities.at[row_index, identity] == 1:
            ind = train_identities.index(identity)
            comment_list.append(train_df_identities.at[row_index, "comment_text"])
            toxic_list.append(train_df_identities.at[row_index, "toxic"])
            augmented.append(0)
            for diff_identity in train_identities[-ind]:
                comment_list.append(train_df_identities.at[row_index, "comment_text"].replace(identity, diff_identity))
                toxic_list.append(train_df_identities.at[row_index, "toxic"])
                augmented.append(1)

100%|██████████| 13947/13947 [00:04<00:00, 3417.72it/s]


In [202]:
data_tuples = list(zip(comment_list, toxic_list))
augmented_data = pd.DataFrame(data_tuples, columns=['comment_text','toxic'])
augmented_data['augmented'] = augmented
augmented_data

,comment_text,toxic,augmented
0,"""\nFair use rationale for Image:Wonju.jpg\n\nT...",0,0
1,"""\nFair use rationale for Image:Wonju.jpg\n\nT...",0,1
2,"""\nFair use rationale for Image:Wonju.jpg\n\nT...",0,1
3,"""\nFair use rationale for Image:Wonju.jpg\n\nT...",0,1
4,"""\nFair use rationale for Image:Wonju.jpg\n\nT...",0,1
...,...,...,...
135544,"""\n Auto guides and the motoring press are not...",0,1
135545,"""\n Auto guides and the motoring press are not...",0,1
135546,"""\n Auto guides and the motoring press are not...",0,1
135547,"""\n Auto guides and the motoring press are not...",0,1


In [207]:
#augmented_data = pd.DataFrame().reindex_like(train_df_short)
#augmented_data
# for row_index in tqdm(range(42,44)):
#     print(train_df_short.where(train_df_short[row_index] == 1))

AttributeError: 'DataFrame' object has no attribute 'rows'

In [205]:
augmented_data.to_csv(Path("./data/train_df_synthetic.csv"))